# Give me some credit dataset

-------------------------------------------------------------------------
       Matthieu
--------------------------------------------------------------------------

## Chargement des librairies

Pour cette analyse nous allons avoir besoin des librairies de calculs (panda et numpy), de graphiques (matplotlib et seaborn) et de scikit learn pour le machine learning.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Scikit learn librairies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

%matplotlib inline

Nous chargons les bases de données dont nous allons avoir besoin.

In [ ]:
df_test= pd.read_csv("../input/give-me-some-credit-dataset/cs-test.csv")
df= pd.read_csv("../input/give-me-some-credit-dataset/cs-training.csv")
dftmp=pd.read_csv("../input/give-me-some-credit-dataset/cs-training.csv")
sample_entry=pd.read_csv("../input/give-me-some-credit-dataset/sampleEntry.csv")

Commencons par afficher les dimensions de la base de donnée.

In [ ]:
df.shape

Puis les 10 premiers éléments du dataset.

In [ ]:
df.head(10)

Enfin une description mathématique des données.

In [ ]:
df.describe()

Regardons à présent les 5 premières valeurs du dataframe sample_entry.csv

In [ ]:
sample_entry.head()

Il s'agit de la probabilité de défaut calculé par la banque pour chaque individu. Nous n'utiliserons évidement pas cette donnée puisqu'elle a déjà été calculée par la banque elle même !

Affichons les colonnes de notre base.

In [ ]:
df.columns

Nous nous intéressons au type de chaque variable.

In [ ]:
df.dtypes.value_counts()

Il n'y a que des valeurs numériques, cela facilitera grandement le traitement de la base.

Regardons maintenant le nombre de valeurs nulles par colonne

In [ ]:
df.isnull().sum()

Nous remarquons quelque chose de problématique. Il manque près de 30 000 valeurs pour le salaire, soit environ 20% des valeurs. Il est pourtant évident que cette donnée est cruciale pour réussir notre classification. Après plusieurs essais, nous en avons conclu que la méthode de la médiane était la plus efficace pour gérer ces valeurs manquantes. Nous effectuerons cette opération par la suite.

Afin d'avoir un vue claire de la répartition des données, nous avons écrit une fonction afin d'afficher la valeur de chaque cellule pour chaque catégorie.

In [ ]:
#A function to print every graph with the ID as 
def print_all_values():
    df1=df.drop('Unnamed: 0',axis=1)
    cols=df1.columns
    for col in cols:
        if (df[col].dtypes !='object'):

            fig1=plt.figure()
            ax1=plt.axes()
            plt.scatter(df[[col]],df['Unnamed: 0'],alpha=1,s=0.5)
            plt.title(col)
            ax1 = ax1.set(xlabel=col, ylabel='ID')
            plt.show()
            
            
print_all_values()

Nous pouvons supprimer les valeurs "aberrantes" de la base de donnée. Cela nous aidera à améliorer les prédictions de nos modèles. Pour nous aider dans cette tâche nous avons créé une fonction qui supprime les valeurs qui paraissent trop extrêmes. Elle prend en paramètre la limite maximum qu'une valeur peut prendre en fonction des autres valeurs du même feature.

In [ ]:
print(df.shape)
def delete_absurd_values(df_transformed,cols,max_value,percentage):
        
        
        for col in cols:
            if (df_transformed[col].dtypes !='object'):
                       
                q99=df_transformed[col].quantile(q=percentage)
                q01=df_transformed[col].quantile(q=(1-percentage))
                for i in df_transformed.index:
                    
                    if (df_transformed.loc[i,col]> max_value*q99 or df_transformed.loc[i,col]< q01/max_value):
                        df_transformed=df_transformed.drop(index=i)
        
        return df_transformed

cols=['DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines',
       'NumberOfDependents']
df=delete_absurd_values(df,cols,4,0.999)
print(df.shape)

On fini par supprimer manuellement les valeurs qui parraissent toujours aberrantes.

In [ ]:
df=df[df.RevolvingUtilizationOfUnsecuredLines <30000]
df=df[df.DebtRatio <100000]
df=df[df.MonthlyIncome <15000000]
df=df[df.NumberRealEstateLoansOrLines <40]

En ce qui concerne les valeurs manquantes, comme nous l'avons dit précédement, nous avons décidé de les remplacer par la valeur médiane de chaque classe. Avant cela, nous avons essayé plusieurs techniques, parmis elles : de les supprimer, supprimer la colonne, remplacer par la moyenne, remplacer par un constante.

In [ ]:
df.fillna(df.median(), inplace=True)

On vérifie qu'il ne reste plus de valeur nulles

In [ ]:
df.isnull().sum()

Vérifions que la répartition de la target est inchangée

In [ ]:
fig11=plt.figure()
ax11=plt.axes()
the_target = dftmp['SeriousDlqin2yrs']
the_target.replace(to_replace=[1,0], value= ['YES','NO'], inplace = True)
plt.title('Target repartition')
ax11 = ax11.set(xlabel='Default proportion', ylabel='Number of people')
the_target.value_counts().plot.pie(startangle=90, autopct='%1.1f%%')
plt.show()

#### Graphiques comparés

In [ ]:
sns.set(style = 'whitegrid', context = 'notebook', rc={'figure.figsize':(20,15)})


cols = ['SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']

#sns.pairplot(df[cols])
plt.show()

Le nombre important de features rend difficile une lecture intelligible de ce graphique. Nous préférerons utiliser une matrice de corrélation pour comprendre le lien entre les features.

#### Matrice de Corrélation

In [ ]:
#Correlation Matrix calcul
corr_mat = df.corr()

fig2=plt.figure()
sns.set(rc={'figure.figsize':(25,15)})
k = 20
cols = corr_mat.nlargest(k, 'SeriousDlqin2yrs')['SeriousDlqin2yrs'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.title('Correlation Matrix')
plt.show()

Grâce à cette matrice de corrélation on remarque que les features qui ont la plus grande influence sur la target sont l'age et d'avoir fait défaut pas le passer. Etonnement le salaire ne semble pas à première vue avoir un grand impact. Cependant ces coefficients restent relativement faibles.

On notera également qu'il y a un très grande corrélation entre les differents temps de défaut de crédit.

## Séparer les données entre les donnée d'entrainement et de test

In [ ]:
X = df.drop('SeriousDlqin2yrs',axis=1)
y = df['SeriousDlqin2yrs']  

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

On utilise 80% du dataset pour entrainer notre algorithme et 20% pour effectuer les tests. On donne un random_state pour qu'à chaque execution de la fonction, celle-ci sépare les données de la meme facon et ainsi on introduit un biais constant à chaque itération.

---------------------------------------------------------------------------------------------
#                  Algorithmes de Machine Learning
----------------------------------------------------------------------------------------------

## Regression Logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100,random_state=0)
logisticRegr.fit(X_train, y_train)

#### Score et erreur

In [ ]:
#ERROR
error = (1 - logisticRegr.score(X_test, y_test))*100
print('Score  = ',logisticRegr.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

## LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis(solver='svd',shrinkage=None,store_covariance=True)
lda.fit(X_train, y_train)

#### Score et erreur

In [ ]:
#ERROR
error = (1 - lda.score(X_test, y_test))*100
print('Score  = ',lda.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, oob_score=True, random_state=0)
rf.fit(X_train,y_train)

#### Score et erreur

In [ ]:
error = (1 - rf.score(X_test, y_test))*100
print('Score  = ',rf.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

## Tree Decision Classifier

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

#### Score et erreur

In [ ]:
error = (1 - clf.score(X_test, y_test))*100
print('Score  = ',clf.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

# Résultats

Après entrainement de nos algorithmes, nous avons obtenu les résultats suivants sur le jeu de test:

In [ ]:
print('Taux de réussite par modèle:\n\nRégression Logistique:',logisticRegr.score(X_test, y_test)*100,'%','\n\nLDA:',lda.score(X_test, y_test)*100,'%','\n\nRandom Forest Classifier:',rf.score(X_test, y_test)*100,'%','\n\nDecision Tree Classifier:',clf.score(X_test, y_test)*100,'%')